In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam


2025-07-18 11:53:31.445563: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-18 11:53:31.446374: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 11:53:31.450272: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 11:53:31.460079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752864811.477269  149878 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752864811.48

# Preprocessing and Preparation

In [ ]:
data_dir = 'data/RecyclableAndHouseholdWasteClassification/images'

image_size = (128, 128) # also try training using 256x256 images
batch_size = 32 # change this to 16, 32, 64, or 128 and compare the results

"""
32: Validation Loss: 1.9282389879226685, Validation Accuracy: 0.6756666898727417
64: Validation Loss: 1.9139728546142578, Validation Accuracy: 0.6703333258628845
"""


datagen = ImageDataGenerator(rescale=1./255, validation_split=0.05) # change to 0.05 for testing image uploads. if we want to compare the accuracy with different settings, change to 0.2.

# load train data (the 80% part of the split)

train_data = datagen.flow_from_directory(data_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', subset='training')

valid_data = datagen.flow_from_directory(data_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', subset='validation')

Found 14250 images belonging to 30 classes.
Found 750 images belonging to 30 classes.


# Build the Model

In [3]:
# the best way to improve accuracy would probably be to create more convolutional layers:

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(len(train_data.class_indices), activation='softmax'))

/home/adi/Congressional-App-Challenge-Backend/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-18 11:53:35.445145: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


# Compile the Model

In [4]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the Model

In [5]:
# Train the model
history = model.fit(
    train_data,
    epochs=14,  # play with this number to maximize accuracy
    validation_data=valid_data
)


/home/adi/Congressional-App-Challenge-Backend/venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/14
446/446 ━━━━━━━━━━━━━━━━━━━━ 50s 109ms/step - accuracy: 0.1546 - loss: 3.2480 - val_accuracy: 0.4760 - val_loss: 1.8748
Epoch 2/14
446/446 ━━━━━━━━━━━━━━━━━━━━ 48s 108ms/step - accuracy: 0.5576 - loss: 1.5991 - val_accuracy: 0.6493 - val_loss: 1.3237
Epoch 3/14
446/446 ━━━━━━━━━━━━━━━━━━━━ 49s 110ms/step - accuracy: 0.7694 - loss: 0.8251 - val_accuracy: 0.7227 - val_loss: 1.0811
Epoch 4/14
446/446 ━━━━━━━━━━━━━━━━━━━━ 50s 111ms/step - accuracy: 0.8906 - loss: 0.4051 - val_accuracy: 0.7360 - val_loss: 1.2021
Epoch 5/14
446/446 ━━━━━━━━━━━━━━━━━━━━ 48s 108ms/step - accuracy: 0.9372 - loss: 0.2390 - val_accuracy: 0.7867 - val_loss: 1.0455
Epoch 6/14
446/446 ━━━━━━━━━━━━━━━━━━━━ 49s 109ms/step - accuracy: 0.9524 - loss: 0.1911 - val_accuracy: 0.7667 - val_loss: 1.1378
Epoch 7/14
446/446 ━━━━━━━━━━━━━━━━━━━━ 51s 115ms/step - accuracy: 0.9614 - loss: 0.1449 - val_accuracy: 0.7760 - val_loss: 1.1294
Epoch 8/14
446/446 ━━━━━━━━━━━━━━━━━━━━ 48s 107ms/step - accuracy: 0.9634 - loss: 0

# Validation for accuracy
Note: check `validation_split=0.05` in "Preprocessing and Preparation." Use `validation_split=0.2` to get an accuracy score that better reflects the model's performance.  

In [6]:
val_loss, val_acc = model.evaluate(valid_data)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_acc}')

24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.7807 - loss: 1.3683
Validation Loss: 1.4117976427078247, Validation Accuracy: 0.7666666507720947


# Save the model

In [7]:
# code here

# Use the Model to Make a Prediction for a user uploaded Image

In [8]:
# code here